# Computer Vision Project
#### Introduction to Machine learning course | Instructor: Dr. Fatemeh Mirsalehi

<font color='cyan'> Responsible TA: Amirhossein Razlighi </font>

In [ ]:
%pip install torch torchvision numpy matplotlib Pillow

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

## Section 1: Implementing a CNN from scratch! (30 points)

### 1.1 - Forward and Backward (20/30 pts)

__CNN__ s are powerfull tools to work with in the scope of _Computer Vision_. You have seen MLPs before and know how to do forward and backward pass on them. In this part, we want to create a convolutional layer (a simple one, of course!) just to understand well that how a Convolutional Layer works, behind the scenes. Then, we can simply use PyTorch's `Conv2D` layer for the rest of this project, because we understood that how Convolutions work, behind the scene!

__FORWARD PASS__:

In forward pass, you should apply the convolution operation on the input image. The convolution operation is as follows:

$$
\text{output}[i, j] = \sum_{k=0}^{K-1} \sum_{l=0}^{L-1} \text{input}[i+k, j+l] \times \text{kernel}[k, l]
$$

You can see a sample convolution operation (with a $3 \times 3$ kernel) in the following image:
<div style="text-align:center;">
  <img src="./Images/Conv.gif" />
</div>

Please note that, these are samples for you to understand the operation better. For _forward pass_ that you should implement, please notice that you should convolve the kernel along all $C$ channels of the input image. So, the output shape should be $(N, H_{out}, W_{out})$. These values are calculated as follows:

$$
H_{out} = \frac{H_{in} + 2 \times \text{padding}}{\text{stride} - HH} + 1
$$
$$
W_{out} = \frac{W_{in} + 2 \times \text{padding}}{\text{stride} - WW} + 1
$$


where $HH$ and $WW$ are the height and width of the kernel, respectively. __stride__ is the step size of the kernel, and __padding__ is the number of zeros that should be padded to the input image. Please also note that $b$ stands for bias, which is a scalar value that should be added to the output of the convolution operation. (for each kernel)

You can see a more detailed example, below:

<div style="text-align:center;">
  <img src="./Images/conv3.gif" />
</div>

__Backward Pass__:

In backward pass, you should calculate gradient of output with respect to the input image and the kernel. These gradients will be used in _optimization_ to update kernel weights. (You saw something similar, in using MLPs with gradient descent algorithm!). So, we need $d_x$, $d_w$ and $d_b$ in the backward pass. These are calculated as follows:

$$
dx_{padded} [n, :, i * stride: i * stride + HH, j * stride: j * stride + WW] += w[f] \times dout[n, f, i, j]
$$

$$
dw_{f} = \sum_{n=0}^{N - 1} \sum_{i=0}^{H_{out} - 1} \sum_{j=0}^{W_{out} - 1} x_{padded}[n, :, i * stride: i * stride + HH, j * stride: j * stride + WW] \times dout[n, f, i, j]  
$$

$$
db_{f} = \sum_{n=0}^{N - 1} \sum_{i=0}^{H_{out} - 1} \sum_{j=0}^{W_{out} - 1} dout[n, f, i, j]
$$

For better understanding of the backward pass, you can see the following image:

<div style="text-align:center;">
  <img src="./Images/backprop_cs231n.png" />
</div>


In [ ]:
class MyConv:
    def __init__(self, stride, padding):
        self.stride = stride
        self.padding = padding
        self.cache = None
        self.dx = None
        self.dw = None
        self.db = None

    def forward(self, x, w, b):
        N, C, H, W = x.shape
        F, _, HH, WW = w.shape
        pad = self.padding
        stride = self.stride
        H_out = (H + 2 * pad - HH) // stride + 1
        W_out = (W + 2 * pad - WW) // stride + 1

        x_pad = np.pad(x, ((0, 0), (0, 0), (pad, pad), (pad, pad)), mode='constant')
        out = np.zeros((N, F, H_out, W_out))

        for n in range(N):
            for f in range(F):
                for k in range(H_out):
                    for l in range(W_out):
                        out[n, f, k, l] = np.sum(
                            x_pad[n, :, k*stride:k*stride+HH, l*stride:l*stride+WW] * w[f, :, :, :]
                        ) + b[f]

        self.cache = (x, w, b, x_pad)
        return out

    def backward(self, dout):
        x, w, b, x_padded = self.cache

        N, C, H, W = x.shape
        F, _, HH, WW = w.shape
        _, _, H_OUT, W_OUT = dout.shape

        dw = np.zeros_like(w)
        dx = np.zeros_like(x)
        db = np.zeros_like(b)
        dx_padded = np.zeros_like(x_padded)

        for n in range(N):
            for f in range(F):
                for i in range(H_OUT):
                    for j in range(W_OUT):
                        dw[f] += x_padded[n, :, i*self.stride:i*self.stride+HH, j*self.stride:j*self.stride+WW] * dout[n, f, i, j]
                        dx_padded[n, :, i*self.stride:i*self.stride+HH, j*self.stride:j*self.stride+WW] += w[f] * dout[n, f, i, j]
                db[f] += np.sum(dout[n, f, :, :])

        dx = dx_padded[:, :, self.padding:-self.padding, self.padding:-self.padding] if self.padding > 0 else dx_padded

        self.dx = dx
        self.dw = dw
        self.db = db
        return dx, dw, db

In [ ]:
def rel_error(x, y):
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [ ]:
# A simple test for forward pass (DO NOT CHANGE)

x_shape = (2, 3, 4, 4)
w_shape = (3, 3, 4, 4)
x = np.linspace(-0.1, 0.5, num=np.prod(x_shape)).reshape(x_shape)
w = np.linspace(-0.2, 0.3, num=np.prod(w_shape)).reshape(w_shape)
b = np.linspace(-0.1, 0.2, num=3)

out = MyConv(stride=2, padding=1).forward(x, w, b)
correct_out = np.array([[[[[-0.08759809, -0.10987781],
                           [-0.18387192, -0.2109216 ]],
                          [[ 0.21027089,  0.21661097],
                           [ 0.22847626,  0.23004637]],
                          [[ 0.50813986,  0.54309974],
                           [ 0.64082444,  0.67101435]]],
                         [[[-0.98053589, -1.03143541],
                           [-1.19128892, -1.24695841]],
                          [[ 0.69108355,  0.66880383],
                           [ 0.59480972,  0.56776003]],
                          [[ 2.36270298,  2.36904306],
                           [ 2.38090835,  2.38247847]]]]])

# The outputted difference which is printed, should be around 1e-8
print ('Testing conv_forward_naive')
print ('difference: ', rel_error(out, correct_out))

In [ ]:
def eval_numerical_gradient_array(f, x, df, h=1e-5):
  grad = np.zeros_like(x)
  it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
  while not it.finished:
    ix = it.multi_index

    oldval = x[ix]
    x[ix] = oldval + h
    pos = f(x).copy()
    x[ix] = oldval - h
    neg = f(x).copy()
    x[ix] = oldval

    grad[ix] = np.sum((pos - neg) * df) / (2 * h)
    it.iternext()
  return grad

In [ ]:
# A simple test for backward pass (DO NOT CHANGE)

x = np.random.randn(4, 3, 5, 5)
w = np.random.randn(2, 3, 3, 3)
b = np.random.randn(
    2,
)
dout = np.random.randn(4, 2, 5, 5)
conv = MyConv(stride=1, padding=1)

dx_num = eval_numerical_gradient_array(
    lambda x: conv.forward(x, w, b), x, dout
)
dw_num = eval_numerical_gradient_array(
    lambda w: conv.forward(x, w, b), w, dout
)
db_num = eval_numerical_gradient_array(
    lambda b: conv.forward(x, w, b), b, dout
)

out = conv.forward(x, w, b)
dx, dw, db = conv.backward(dout)

# Your printed errors should be around 1e-9
print("Testing conv_backward_naive function")
print("dx error: ", rel_error(dx, dx_num))
print("dw error: ", rel_error(dw, dw_num))
print("db error: ", rel_error(db, db_num))

### 1.2 - Visualizing Convolution results (10/30 pts)

In this part, we try to visualize the results of the convolution operation. We will 2 sample photos (in `Images/Sample`) and then we will use our `MyConv` class to apply some cool convolutions (and see their result on an image).

In [ ]:
from PIL import Image

first_img, second_img = Image.open("./Images/Sample/Sharif_1.JPG"), Image.open(
    "./Images/Sample/Sharif_2.jpg"
)
first_img = first_img.resize((256, 256))
second_img = second_img.resize((256, 256))

In [ ]:
first_img

In [ ]:
second_img

Convolutions have some interesting usages in image processing. For example, you can extract edges from an image by applying a convolution with a kernel that detects edges. Or you can convolve a specific kernel with your image to make it grayscale or blurred! Look at the definitions below:

__Edge Detection Kernel__:

_Sobel_ is one of the most famous edge detection kernels. It has two kernels, one for detecting vertical edges and the other for horizontal edges. You can see the kernels below:
$$
\text{Sobel}_x = \begin{bmatrix}
-1 & 0 & 1 \\
-2 & 0 & 2 \\
-1 & 0 & 1
\end{bmatrix}
$$

$$
\text{Sobel}_y = \begin{bmatrix}
-1 & -2 & -1 \\
0 & 0 & 0 \\
1 & 2 & 1
\end{bmatrix}
$$


__Grayscale Kernel__:

You can simply convolve the image with the following kernel to make it grayscale:

$$
\text{Grayscale\_across\_R} = \begin{bmatrix}
0 & 0 & 0 \\
0 & 0.3 & 0 \\
0 & 0 & 0
\end{bmatrix}
$$

$$
\text{Grayscale\_across\_G} = \begin{bmatrix}
0 & 0 & 0 \\
0 & 0.6 & 0 \\
0 & 0 & 0
\end{bmatrix}
$$

$$
\text{Grayscale\_across\_B} = \begin{bmatrix}
0 & 0 & 0 \\
0 & 0.1 & 0 \\
0 & 0 & 0
\end{bmatrix}
$$

(Convolve each kernel with its corresponding channel)

__Blurring Kernel__:

One of the famous kernels to blur an image is _Gaussian Blur_. You can see the kernel below:

$$
\text{Gaussian\_Blur} = \frac{1}{16} \times \begin{bmatrix}
1 & 2 & 1 \\
2 & 4 & 2 \\
1 & 2 & 1
\end{bmatrix}
$$


In [ ]:
img_size = 256

x = np.zeros((2, 3, img_size, img_size))
x[0] = np.array(first_img).transpose(2, 0, 1)
x[1] = np.array(second_img).transpose(2, 0, 1)

# Defining convolution kernels
sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
sobel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
grayscale_R = np.array([[0, 0, 0], [0, 0.3, 0], [0, 0, 0]])
grayscale_G = np.array([[0, 0, 0], [0, 0.6, 0], [0, 0, 0]])
grayscale_B = np.array([[0, 0, 0], [0, 0.1, 0], [0, 0, 0]])
gaussian_blur = (1/16) * np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]])

# A convolution weight, holding 4 filters 3x3
w = np.stack([
    np.stack([sobel_x, sobel_x, sobel_x]),
    np.stack([sobel_y, sobel_y, sobel_y]),
    np.stack([grayscale_R, grayscale_G, grayscale_B]),
    np.stack([gaussian_blur, gaussian_blur, gaussian_blur])
])

b = np.array([0, 128, 128, 0])  # todo: chatgpt says [0,0,0,0]. what to do?

conv = MyConv(stride=1, padding=1)
out = conv.forward(x, w, b)

In [ ]:
titles = ['Sobel X', 'Sobel Y', 'Grayscale', 'Gaussian Blur']

plt.figure(figsize=(12, 6))
plt.subplot(2, 5, 1)
plt.imshow(first_img)
plt.title('1st Image')
plt.axis('off')
for i in range(4):
    plt.subplot(2, 5, i + 2)
    plt.imshow(out[0, i, :, :], cmap='gray')
    plt.title(titles[i])
    plt.axis('off')
plt.show()

plt.figure(figsize=(12, 6))
plt.subplot(2, 5, 1)
plt.imshow(second_img)
plt.title('2nd Image')
plt.axis('off')
for i in range(4):
    plt.subplot(2, 5, i + 2)
    plt.imshow(out[1, i, :, :], cmap='gray')
    plt.title(titles[i])
    plt.axis('off')
plt.show()

Now that we understood how Convolution works, we proceed further to use it in our models to perform a desired and specific task! We can use PyTorch's `Conv2D` layer from now on, because we understood how Convolutions work, behind the scene!

## Section 2: Neural Style Transfer (90 points)

In this part, we are going to implement a famous technique in the field of _Computer Vision_ called __Neural Style Transfer__. This technique is used to apply the style of a specific image to another image. For example, you can apply the style of a famous painting to your photo! Let's get started!

### 2.1 - Importing Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
from torchvision.models import vgg19, VGG19_Weights

import copy

In [ ]:
device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
torch.set_default_device(device)
device

### 2.2 - Loading Images and doing some Preprocessing (5/90 pts)

In this part, you should load the content and style images and preprocess them. Preprocessing includes resizing the images to a specific size, normalizing them, and converting them to PyTorch tensors. You can use `torchvision.transforms` to do these tasks.

In [ ]:
img_size = 512 if not device == "cpu" else 256

transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
def load_image(image_path, transform):
    return transform(Image.open(image_path).convert('RGB')).unsqueeze(0).to(device)

style_image = load_image('./Images/Neural_Transfer/style.jpeg', transform)
content_image = load_image('./Images/Neural_Transfer/Content.jpeg', transform)

style_image.size(), content_image.size()

Here, you should implement a helper function to show you images (which are now in form of pytorch tensors) in the notebook. You can use `torchvision.transforms.ToPILImage` to convert a tensor to a PIL image.

In [ ]:
def imshow(tensor, title=None):
    image = tensor.cpu().clone()
    image = image.squeeze(0)
    image = transforms.ToPILImage()(image)
    plt.imshow(image)
    if title:
        plt.title(title)
    plt.axis('off')
    plt.show()

In [ ]:
imshow(style_image, "Style Image")

In [ ]:
imshow(content_image, "Content Image")

### 2.2 - VGG19 Model (15/90 pts)

One of the important parts of Neural Style Transfer is to use a pre-trained model to extract features from the content and style images. Here, we are going to use a pre-trained VGG19 model to extract features from the images. You can use `torchvision.models.vgg19` to load the model. You should also freeze the weights of the model to prevent them from updating during the optimization process.

<div style="text-align:center;">
  <img src="./Images/VGG-19.png" />
</div>

But what is VGG model? Let's see with a simple experiment:

In [ ]:
import certifi
import os

os.environ['SSL_CERT_FILE'] = certifi.where()

Here, please load `VGG19` model from `torchvision.models` and see the architecture of the model. You may need models `features` and `classifier` part stored seperately as we will discuss these both parts.

In [ ]:
vgg_model = vgg19(weights=VGG19_Weights.DEFAULT).to(device)
vgg_model_features = vgg_model.features
vgg_model_classifier = vgg_model.classifier

In [ ]:
import torchsummary
torchsummary.summary(vgg_model, input_size=(3, img_size, img_size))

Please briefly explain what is a VGG model used for and what does each `features` and `classifier` part, represent?

**Your Answer:**

The VGG (Visual Geometry Group) model is a convolutional neural network architecture. It is widely used in computer vision tasks such as image classification, feature extraction, object detection, and image segmentation.

- Features: This part consists of a series of convolutional layers followed by max-pooling layers. The primary role of the features part is to extract spatial hierarchies of features from the input images.
- Classifier: This part consists of fully connected layers (aka dense layers). This part's role is to take the high-level features extracted by the convolutional layers and perform the final classification.

Now, we are going to give a sample image `Images/Neural_Transfer/Car.jpeg` to the model, see the output for classification and also visualize the feature maps (outputs of each convolutional layer) to see what's going on!

In [ ]:
vgg_test_image = Image.open("./Images/Neural_Transfer/Car.jpeg")
plt.imshow(vgg_test_image)
plt.title('Test Image')
plt.axis('off')
plt.show()


preprocess_vgg = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
input_tensor = preprocess_vgg(vgg_test_image).unsqueeze(0).to(device)
vgg_model.eval()
with torch.no_grad():
    output = vgg_model(input_tensor)
_, predicted = torch.max(output, 1)
print(f'Predicted class for test image: {predicted}')

As you can see, the output of the classifier is an integer. This integer is actually the **class number** in _Image Net_ dataset. Let's get the name of the class:

In [ ]:
import requests

class_idx = requests.get(
    "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
).json()

# Get the class name
class_name = class_idx[predicted]

print(f"Predicted class: {class_name}")

Now that we understood the architecture of VGG, and also the output of its classifier, it's time to extract features from the images! And see what's going on in the feature maps!

In [ ]:
def get_feature_maps(model, x):
    feature_maps = []
    for layer in model:
        x = layer(x)
        if isinstance(layer, nn.Conv2d):
            feature_maps.append(x)
    return feature_maps

def visualize_feature_maps(feature_maps):
    for i, fm in enumerate(feature_maps):
        fm = fm.squeeze(0)
        plt.figure(figsize=(10, 10))
        for j in range(min(16, fm.shape[0])):
            ax = plt.subplot(4, 4, j + 1)
            plt.imshow(fm[j].cpu().detach().numpy(), cmap='coolwarm')
            ax.axis('off')
        plt.suptitle(f"Feature Maps - Layer {i + 1}")
        plt.show()

feature_maps = get_feature_maps(vgg_model_features, input_tensor)
visualize_feature_maps(feature_maps)

It's fun too look under the hood, isn't it? As you can see, we can guess what are the learned features in each layer if the network (at least in the first layers). For example, in the first layer, the network learns to detect edges, and in the second layer, it learns to detect simple shapes. In the deeper layers, the network learns to detect more complex features.

### 2.3 - Content and Style Loss (15/90 pts)

In order to learn the correct image and keep the content of the original image while applying the style of the seconf image, we need 2 losses: 1. Content Loss and 2. Style Loss. Let's delve into each one!

**Content Loss** is responsible for keeping the content of the original image. It is calculated as the mean squared error between the feature maps of the original image and the generated image.

In [ ]:
class ContentLoss(nn.Module):
    def __init__(self, target):
        super(ContentLoss, self).__init__()
        self.target = target.detach()
        self.loss = None

    def forward(self, input_f):
        self.loss = nn.functional.mse_loss(input_f, self.target)
        return input_f

**Style Loss** is responsible for applying the style of the second image to the generated image. It is calculated as the mean squared error between the Gram matrices of the feature maps of the second image and the generated image.

In [ ]:
def gram_calculator(input_f):
    """
    Calculate the normalized Gram Matrix of a given input
    """
    a, b, c, d = input_f.size()
    features = input_f.view(a * b, c * d)
    G = torch.mm(features, features.t())
    return G.div(a * b * c * d)

In [ ]:
class StyleLoss(nn.Module):
    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = gram_calculator(target_feature).detach()
        self.loss = None

    def forward(self, input_f):
        self.loss = nn.functional.mse_loss(gram_calculator(input_f), self.target)
        return input_f

Now, we use `features` part of the VGG model to extract features from the content and style images. Then, we calculate the content and style loss using the extracted features.

In [ ]:
cnn = vgg19(weights=VGG19_Weights.DEFAULT).features.to(device).eval()

In [ ]:
cnn_normalization_mean = torch.tensor(
    [0.485, 0.456, 0.406]
).to(device)  # Normalization mean for the VGG19 model
cnn_normalization_std = torch.tensor(
    [0.229, 0.224, 0.225]
).to(device)  # Normalization std for the VGG19 model

# Normalize the images in order to feed it to the VGG19 model
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=cnn_normalization_mean, std=cnn_normalization_std)
])

content_image = load_image('./Images/Neural_Transfer/Content.jpeg', transform)
style_image = load_image('./Images/Neural_Transfer/style.jpeg', transform)

Please note that after which layers you should calculate the content loss and after which ones, you should calculate the style loss. Use the layer names you saw earlier in the `torchsummary` output for VGG19 model.

In [ ]:
content_layers_default = ['conv_21']
style_layers_default = ['conv_1', 'conv_6', 'conv_11', 'conv_20', 'conv_29']

def get_features(image, model, layers):
    features = {}
    x = image
    for name, layer in model._modules.items():
        x = layer(x)
        if name in layers:
            features[layers[name]] = x
    return features

layer_map = {
    '0': 'conv_1',
    '5': 'conv_6',
    '10': 'conv_11',
    '19': 'conv_20',
    '21': 'conv_21',
    '28': 'conv_29'
}

content_features = get_features(content_image, cnn, layer_map)
style_features = get_features(style_image, cnn, layer_map)

content_loss_layers = [ContentLoss(content_features[layer]) for layer in content_layers_default]
style_loss_layers = [StyleLoss(style_features[layer]) for layer in style_layers_default]

In [ ]:
model = nn.Sequential()
i = 0
content_losses = []
style_losses = []

for layer in cnn.children():
    if isinstance(layer, nn.Conv2d):
        i += 1
        name = f"conv_{i}"
    elif isinstance(layer, nn.ReLU):
        name = f"relu_{i}"
        layer = nn.ReLU(inplace=False)
    elif isinstance(layer, nn.MaxPool2d):
        name = f"pool_{i}"
    elif isinstance(layer, nn.BatchNorm2d):
        name = f"bn_{i}"
    else:
        raise RuntimeError(f'Unrecognized layer: {layer.__class__.__name__}')
    model.add_module(name, layer)
    if name in content_layers_default:
        target = model(content_image).detach()
        content_loss = ContentLoss(target)
        model.add_module(f"content_loss_{i}", content_loss)
        content_losses.append(content_loss)
    if name in style_layers_default:
        target_feature = model(style_image).detach()
        style_loss = StyleLoss(target_feature)
        model.add_module(f"style_loss_{i}", style_loss)
        style_losses.append(style_loss)

for i in range(len(model) - 1, -1, -1):
    if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
        break

model = model[:i+1]

plt.imshow(Image.open('./Images/Neural_Transfer/Content.jpeg').convert('RGB'))
plt.title('Input Image')
plt.axis('off')
plt.show()

(15/90 pts) Now, we need to define our optimizer. We use `LBFGS` optimizer. Let's get familiar with this optimizer a little bit:

Let's use this optimizer to optimize a simple function $f(x) = x^2$. We want to find the minimum of this function using `LBFGS` optimizer. First, you should initialize a tensor $x$ with a random value. Then, you should define a closure function that calculates the value of the function and its gradient. Finally, you should use the `step` method of the optimizer to update the value of $x$.

And lastly, you should generate an animation using `matplotlib.animation` to show how the value of $x$ changes during optimization.

In [ ]:
import matplotlib.animation as animation
from torch.optim import LBFGS
import IPython.display as display
from random import uniform


def quadratic_function(x):
    return x**2

x = torch.tensor([uniform(-10, 10)], requires_grad=True)
optimizer = LBFGS([x])


# Closure function and optimization loop
x_values = []
y_values = []

def closure():
    optimizer.zero_grad()
    y = quadratic_function(x)
    y.backward()
    x_values.append(x.item())
    y_values.append(y.item())
    return y

for _ in range(20):
    optimizer.step(closure)


# Animation
fig, ax = plt.subplots()
ax.set_xlim(-11, 11)
ax.set_ylim(0, 101)
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.set_title('Optimization Process of f(x)')
line, = ax.plot([], [], lw=2)
point, = ax.plot([], [], 'ro')

def init():
    line.set_data([], [])
    point.set_data([], [])
    return line, point

def animate(i):
    x_vals = x_values[:i+1]
    y_vals = y_values[:i+1]
    line.set_data(x_vals, y_vals)
    point.set_data(x_vals[-1], y_vals[-1])
    return line, point

plot_animation = animation.FuncAnimation(fig, animate, init_func=init, frames=len(x_values), interval=200, blit=True)
plt.close()
display.HTML(plot_animation.to_jshtml())

(35/90 pts) Now, let's use this optimizer to optimize our `input_image` to minimize the content and style loss. You should define a closure function that calculates the content and style loss and their gradients. Then, you should use the `step` method of the optimizer to update the value of the `input_image`. Please also **note** that you should use a weighted sum of the content and style loss to optimize the input image. (weight of each loss is a hyperparameter)

In [ ]:
# Train your model using LBFGS optimizer and 2 implemented losses (style loss and content loss)
# You should report the `style loss`, `content loss`, and `total loss` for each iteration. Note that `total loss`
# is the sum of `style loss` and `content loss` with some weights (alpha and beta) which you should tune them.
# Also, please plot the image each 50 iterations.
# An important note is that your optimizer might change the values of the image tensor in a way that the pixel values
# are no longer in the range of [0, 1] (in normalized format). So, you should clip the values of the image tensor
# to be in the range of [0, 1] after each iteration, to prevent any problems occuring to your image.


def get_style_model_and_losses(
    cnn,
    content_image,
    style_image,
    content_layers=content_layers_default,
    style_layers=style_layers_default
):
    content_losses = []
    style_losses = []
    model = nn.Sequential()
    i = 0
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = f'conv_{i}'
        elif isinstance(layer, nn.ReLU):
            name = f'relu_{i}'
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = f'pool_{i}'
        elif isinstance(layer, nn.BatchNorm2d):
            name = f'bn_{i}'
        else:
            raise RuntimeError(f'Unrecognized layer: {layer.__class__.__name__}')
        model.add_module(name, layer)
        if name in content_layers:
            target = model(content_image).detach()
            content_loss = ContentLoss(target)
            model.add_module(f"content_loss_{i}", content_loss)
            content_losses.append(content_loss)
        if name in style_layers:
            target_feature = model(style_image).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module(f"style_loss_{i}", style_loss)
            style_losses.append(style_loss)
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break
    model = model[:i + 1]
    return model, content_losses, style_losses


content_image = load_image('./Images/Neural_Transfer/Content.jpeg', transform)
style_image = load_image('./Images/Neural_Transfer/style.jpeg', transform)
model, content_losses, style_losses = get_style_model_and_losses(cnn, content_image, style_image)
content_losses_list = []
style_losses_list = []
total_losses_list = []

content_image = load_image('./Images/Neural_Transfer/Content.jpeg', transform)
style_image = load_image('./Images/Neural_Transfer/style.jpeg', transform)
input_img = content_image.clone().requires_grad_(True)
optimizer = optim.LBFGS([input_img])


# Training loop
num_steps = 151
alpha = 10**5
beta = 10**1

for i in range(num_steps):
    def closure():
        input_img.data.clamp_(0, 1)
        optimizer.zero_grad()
        model(input_img)

        content_score = sum([l.loss for l in content_losses])
        style_score = sum([l.loss for l in style_losses])
        content_score *= alpha
        style_score *= beta
        total_loss = style_score + content_score
        # total_loss = content_score * alpha + style_score * beta
        total_loss.backward()
        content_losses_list.append(content_score)
        style_losses_list.append(style_score.item())
        total_losses_list.append(total_loss.item())

        if i % 50 == 0:
            print(f'Iteration {i}:')
            print(f'Style Loss: {style_score.item()}')
            print(f'Content Loss: {content_score}')
            print(f'Total Loss: {total_loss.item()}')
            imshow(input_img, f'Step {i}')
        return total_loss

    optimizer.step(closure)

Now, let's see the result of optimization process on our `input_image`.

In [ ]:
  input_img.data.clamp_(0, 1)
imshow(input_img, title='Final Image')

(5/90 pts) Also, let's plot our `style_losses`, `content_losses` and `total_losses` to see how they change during the optimization process.

In [ ]:
#TODO: Plot the total weighted loss w.r.t. the iteration number
plt.figure(figsize=(10, 5))
plt.plot(total_losses_list)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Total Loss during optimization')
plt.legend()
plt.show()

In [ ]:
# TODO: Plot the style loss w.r.t. the iteration number
plt.figure(figsize=(10, 5))
plt.plot(style_losses_list)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Style Loss during optimization')
plt.legend()
plt.show()

In [ ]:
# TODO: Plot the content loss w.r.t. the iteration number
plt.figure(figsize=(10, 5))
plt.plot(content_losses_list)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Content Loss during optimization')
plt.legend()
plt.show()